In [3]:
import math

In [101]:
class Thread:
    def __init__(self, index, p = 0):
        self.index = index
        self.p = p # priority
        
    def __repr__(self):
        return f'({self.index}, {self.p})'

In [133]:
# Jobs are implemetned as FIFO. I just take and serve the first element in the list
class MinHeap:
    def __init__(self, ary, max_size):
        self.heap = ary
        self.size = len(ary)
        self.max_size = max_size
        
    def left_child(self, i):
        """
        Args:
            i (int): index of the parent of the left node of interest
        Returns:
            2*i + 1 (int): index of left child
            
        NOTE: correction of plus 1 is needed since python start counting from 0 indices
        """
        return 2*i + 1
    
    def right_child(self, i):
        """
        NOTE: correction of plus 1 is needed since python start counting from 0 indices
        """
        return 2*i + 2
    
    def parent(self, i):
        """
        NOTE: correction of minus 1 is needed since python start counting from 0 indices
        """
        if i == 0:
            return 0
        if i % 2 == 0:
            return math.floor(i/2) - 1
        return math.floor(i/2)
        
    def sift_down(self, i, swaps = []):
        
        # Initialize
        min_index = i
        
        # Swap with minimum
        left = self.left_child(i) #2
        if left < self.size and self.heap[left].p < self.heap[min_index].p:
            # update index
            min_index = left
            
        # Swap with minimum
        right = self.right_child(i)
        if right < self.size and self.heap[right].p < self.heap[min_index].p:
            # update index
            min_index = right
            
        if min_index == i:
            if left < self.size and self.heap[left].index < self.heap[min_index].index:
                min_index = left
            if right < self.size and self.heap[right].index < self.heap[min_index].index:
                min_index = right
            
        # recursive call on min_index if one of the 2 if above is satisfied
        if min_index != i:
            # update swaps logger
            swaps.append((i, min_index))
            
            # swap
            self.heap[i], self.heap[min_index] = self.heap[min_index], self.heap[i]
            self.sift_down(min_index, swaps)
            
        # return swaps of a single node sifted down
        return swaps
    
    
    def sift_up(self, i):
        # Easier than sift down, since I have one parent
        condition_on_idx = self.heap[self.parent(i)].p == self.heap[i].p and self.heap[self.parent(i)].index > self.heap[i].index
        condition_on_p = self.heap[self.parent(i)].p > self.heap[i].p
        while condition_on_p or condition_on_idx:
            # swap, update index, repeat
            parent_idx = self.parent(i)
            self.heap[parent_idx], self.heap[i] = self.heap[i], self.heap[parent_idx]
            i = self.parent(i)
        

    def build_heap(self):
        """Build a min heap inplace.
        
        Returns a sequence of swaps performed by the algorithm.
        """
        swaps = []
        for i in reversed(range(0, math.floor(self.size/2))):
            swaps = self.sift_down(i, swaps)
            
        return swaps
    
    
    def change_priority(self, i, p):
        """
        Args:
            i (int): index of node which priority must be changed
            p (int): new priority
        """
        old_priority = self.heap[i].p
        self.heap[i].p = p
        if old_priority > p:
            self.sift_up(i)
        else:
            self.sift_down(i)
            
    def get_min(self):
        # Top operation, not pop
        return self.heap[0] # root
    
    # The issue is to get thread priority.
    # In fact, this is done by a min operation in the boiler plate ==> O(n)
    # Instead, I can make it log(n) by extract_min of a min heap.
    # After assigning, I change the thread priority, which is itself O(log(n))
    # Done

In [135]:
# python3

from collections import namedtuple

AssignedJob = namedtuple("AssignedJob", ["worker", "started_at"])


def assign_jobs(n_workers, jobs):
    # All priorities initialized at the same level
    threads = MinHeap([Thread(i, 0) for i in range(n_workers)], n_workers)
    threads.build_heap()
    result = []
    for job in jobs:
        next_worker = threads.get_min()
        result.append(AssignedJob(next_worker.index, next_worker.p))
        threads.change_priority(0, next_worker.p + job)

    return result


def main():
    n_workers, n_jobs = map(int, input().split())
    jobs = list(map(int, input().split()))
    assert len(jobs) == n_jobs

    assigned_jobs = assign_jobs(n_workers, jobs)

    for job in assigned_jobs:
        print(job.worker, job.started_at)


if __name__ == "__main__":
    main()

 2 5
 1 2 3 4 5


0 0
1 0
0 1
1 2
0 4
